In [2]:
import xarray as xr
import numpy  as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pymannkendall as mk

In [3]:
# time series dir: /raid52/yycheng/MPAS/REFERENCE/CMORPH/DAILY/spatialmean
# year   = "2003"
dirin  = "/raid52/yycheng/MPAS/REFERENCE/CN05.1/1961-2014" 
filein = "sel_CM05.1.nc"
ds_disk = xr.open_dataset(dirin + "/" + filein)
ds_disk

<xarray.Dataset>
Dimensions:  (lat: 160, lon: 280, time: 8262)
Coordinates:
  * time     (time) datetime64[ns] 1961-04-01 1961-04-02 ... 2014-08-31
  * lon      (lon) float32 70.125 70.375 70.625 ... 139.375 139.625 139.875
  * lat      (lat) float32 15.125 15.375 15.625 15.875 ... 54.375 54.625 54.875
Data variables:
    pre      (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    Conventions:  CF-1.4
    history:      Thu Jan 21 18:46:56 2021: cdo -L -remapbil,grid_info -selmo...
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...

In [97]:
# 使用pandas的时间索引来处理，创建，并再ds中引用

# 直接用数字来选取
ds_disk['pre'].sel(time="1964")

# 创建pd时间序列来选取
time_idx_long = pd.date_range(start = "1961-04-01", end = "2018-08-01",freq='d') # dtype datetime64
time_idx_short = pd.date_range(start = "2014-04-01", end = "2014-08-01",freq='d') # dtype datetime64
# time_idx = pd.period_range(start = "2004-04-01", end = "2004-08-01",freq='m') # dtype period

# 直接用 time_idx 进行索引,这里使用一个稍短的数组来尝试
ds_disk['pre'].sel(time = time_idx_short)

# 也可以直接转换成 pandas 的日期格式，然后进行大小的比较
pd_time = pd.to_datetime(ds_disk.time.values)
time_idx_short_loc = (pd_time > "2014-04-01") & (pd_time < "2014-08-01")
ds_disk['pre'][time_idx_short_loc]

<xarray.DataArray 'pre' (time: 121, lat: 160, lon: 280)>
[5420800 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2014-04-02 2014-04-03 ... 2014-07-31
  * lon      (lon) float32 70.125 70.375 70.625 ... 139.375 139.625 139.875
  * lat      (lat) float32 15.125 15.375 15.625 15.875 ... 54.375 54.625 54.875
Attributes:
    long_name:  CN05.1 daily, precipitation,mm/day                           ...

In [106]:
ds_disk.time # 直接查看变量的坐标 （ 用属性的方法查询 time.dt 大概是每个时间的拆分方法，具体到年月日时分秒
ds_disk.time.dt.year
ds_disk.time.dt.month
ds_disk.time.dt.day
ds_disk.time.dt.hour
ds_disk.time.dt.minute
ds_disk.time.dt.second

<xarray.DataArray 'second' (time: 8262)>
array([0, 0, 0, ..., 0, 0, 0])
Coordinates:
  * time     (time) datetime64[ns] 1961-04-01 1961-04-02 ... 2014-08-31

In [118]:
# 设置一个 dict 存放进行数据选取的相关信息，然后用 dict 进行索引
# for i in ds_disk['time.year']:
lc  = ds_disk.lon
la  = ds_disk.lat
time= ds_disk.time
# ds_disk.coords
select_dict = {}
# select_dict = dict(time = (ds_disk.time.dt.year==2003) & (ds_disk.time.dt.month==4))
# time 使用这个也可
# select_dict['time']=time[(time.dt.year==2003) & (time.dt.month==4)]

select_dict['time']=(ds_disk.time.dt.year==2003) & (ds_disk.time.dt.month==4)
select_dict['lon'] = lc[(lc > 70) & (lc < 110)]
select_dict['lat'] = la[(la > 20) & (la < 60) ]
# select_dict 存放的是确切的值，因此 对于坐标是 name 索引， 对于 index lookup 是值索引
ds_disk['pre'].loc[select_dict]

<xarray.DataArray 'pre' (time: 30, lat: 140, lon: 160)>
[672000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2003-04-01 2003-04-02 ... 2003-04-30
  * lon      (lon) float32 70.125 70.375 70.625 ... 109.375 109.625 109.875
  * lat      (lat) float32 20.125 20.375 20.625 20.875 ... 54.375 54.625 54.875
Attributes:
    long_name:  CN05.1 daily, precipitation,mm/day                           ...

In [127]:
# 下面是一个用位置进行索引的例子，这里的 select_dict_isel 保存的就是位置的信息
select_dict_isel = {}
select_dict_isel['time'] = (ds_disk.time.dt.year==2003) & (ds_disk.time.dt.month==4)
select_dict_isel['lon'] = (lc > 70) & (lc < 110)
select_dict_isel['lat'] = (la > 20) & (la < 60) 
ds_disk['pre'][select_dict_isel]
ds_disk['pre'].isel(select_dict_isel)
# select_dict_isel

<xarray.DataArray 'pre' (time: 30, lat: 140, lon: 160)>
[672000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2003-04-01 2003-04-02 ... 2003-04-30
  * lon      (lon) float32 70.125 70.375 70.625 ... 109.375 109.625 109.875
  * lat      (lat) float32 20.125 20.375 20.625 20.875 ... 54.375 54.625 54.875
Attributes:
    long_name:  CN05.1 daily, precipitation,mm/day                           ...

In [112]:
# 尝试一下groupby
ds_disk.groupby('lat')


DatasetGroupBy, grouped over 'lat' 
160 groups with labels 15.125, 15.375, ..., 54.875.